In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from functools import partial
from time import time
from objective_functions import arctan_loss
from utils import compute_crossing_percentage, scoring_loss, plot_quantiles, pinball_loss_total
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [11]:
dataset = pd.read_csv('dataset_complete_fillna.csv')

In [3]:
dataset['A_1_Moment'] = dataset['A_1_Moment'] * 100
dataset['B_1_Moment'] = dataset['B_1_Moment'] * 100
dataset['C_1_Moment'] = dataset['C_1_Moment'] * 100
dataset['A_2_Moment'] = dataset['A_2_Moment'] * 1000
dataset['B_2_Moment'] = dataset['B_2_Moment'] * 1000
dataset['C_2_Moment'] = dataset['C_2_Moment'] * 1000
dataset['A_3_Moment'] = dataset['A_3_Moment'] * 1000
dataset['B_3_Moment'] = dataset['B_3_Moment'] * 1000
dataset['C_3_Moment'] = dataset['C_3_Moment'] * 1000
dataset['A_4_Moment'] = dataset['A_4_Moment'] * 100000
dataset['B_4_Moment'] = dataset['B_4_Moment'] * 100000
dataset['C_4_Moment'] = dataset['C_4_Moment'] * 100000
dataset['A_5_Moment'] = dataset['A_5_Moment'] * 1000000
dataset['B_5_Moment'] = dataset['B_5_Moment'] * 1000000
dataset['C_5_Moment'] = dataset['C_5_Moment'] * 1000000
dataset['A_6_Moment'] = dataset['A_6_Moment'] * 10000000
dataset['B_6_Moment'] = dataset['B_6_Moment'] * 10000000
dataset['C_6_Moment'] = dataset['C_6_Moment'] * 10000000
dataset['A_7_Moment'] = dataset['A_7_Moment'] * 100000000
dataset['B_7_Moment'] = dataset['B_7_Moment'] * 100000000
dataset['C_7_Moment'] = dataset['C_7_Moment'] * 100000000

dataset['A_order_pair_1_Moment'] = dataset['A_order_pair_1_Moment'] * 10000
dataset['B_order_pair_1_Moment'] = dataset['B_order_pair_1_Moment'] * 10000
dataset['C_order_pair_1_Moment'] = dataset['C_order_pair_1_Moment'] * 10000
dataset['A_order_pair_2_Moment'] = dataset['A_order_pair_2_Moment'] * 1000000
dataset['B_order_pair_2_Moment'] = dataset['B_order_pair_2_Moment'] * 1000000
dataset['C_order_pair_2_Moment'] = dataset['C_order_pair_2_Moment'] * 1000000
dataset['A_order_pair_3_Moment'] = dataset['A_order_pair_3_Moment'] * 10000000
dataset['B_order_pair_3_Moment'] = dataset['B_order_pair_3_Moment'] * 10000000
dataset['C_order_pair_3_Moment'] = dataset['C_order_pair_3_Moment'] * 10000000
dataset['A_order_pair_4_Moment'] = dataset['A_order_pair_4_Moment'] * 100000000
dataset['B_order_pair_4_Moment'] = dataset['B_order_pair_4_Moment'] * 100000000
dataset['C_order_pair_4_Moment'] = dataset['C_order_pair_4_Moment'] * 100000000
dataset['A_order_pair_5_Moment'] = dataset['A_order_pair_5_Moment'] * 10000000000
dataset['B_order_pair_5_Moment'] = dataset['B_order_pair_5_Moment'] * 10000000000
dataset['C_order_pair_5_Moment'] = dataset['C_order_pair_5_Moment'] * 10000000000
dataset['A_order_pair_6_Moment'] = dataset['A_order_pair_6_Moment'] * 100000000000
dataset['B_order_pair_6_Moment'] = dataset['B_order_pair_6_Moment'] * 100000000000
dataset['C_order_pair_6_Moment'] = dataset['C_order_pair_6_Moment'] * 100000000000
dataset['A_order_pair_7_Moment'] = dataset['A_order_pair_7_Moment'] * 1000000000000
dataset['B_order_pair_7_Moment'] = dataset['B_order_pair_7_Moment'] * 1000000000000
dataset['C_order_pair_7_Moment'] = dataset['C_order_pair_7_Moment'] * 1000000000000

In [4]:
train_val_data = dataset[dataset['Group']=='Spring']
test_data = dataset[dataset['Group']=='Fall']

import warnings
warnings.filterwarnings("ignore")

In [5]:
for fea in range(54,55):
    print('# fea', fea)
    r2_valid = []

    acc_score_train = []
    Precision_train = []
    Recall_train = []
    F1_train = []

    acc_score_valid = []
    Precision_valid = []
    Recall_valid = []
    F1_valid = []

    for i in range(5):
        train_data = pd.concat([train_val_data[:i*50], train_val_data[i*50+49:]], axis=0)
        valid_data = train_val_data[i*50:i*50+49]

        train_X = train_data.drop(['SubjectID','Group','exam_avg'], axis=1).values
        train_Y = train_data[['exam_avg']].values

        train_Y_mean = np.mean(train_Y)

        valid_X = valid_data.drop(['SubjectID','Group','exam_avg'], axis=1).values
        valid_Y = valid_data[['exam_avg']].values

        valid_Y_mean = np.mean(valid_Y)

        test_X = test_data.drop(['SubjectID','Group','exam_avg'], axis=1).values
        test_Y = test_data[['exam_avg']].values
        test_Y_mean = np.mean(test_Y)

        import numpy as np
        from sklearn.feature_selection import SelectKBest
        from sklearn.feature_selection import f_regression

        #test_X_selected = (test_X_selected  - np.mean(test_X_selected , axis=0)) / np.std(test_X_selected , axis=0)

        selector = SelectKBest(score_func=f_regression, k=fea)

        selector.fit(train_X, train_Y)

        selected_features = selector.get_support(indices=True)

        train_X_selected = selector.transform(train_X)
        valid_X_selected = selector.transform(valid_X)
        test_X_selected = selector.transform(test_X)

        #train_X_selected = (train_X_selected  - np.mean(train_X_selected , axis=0)) / np.std(train_X_selected , axis=0)
        #valid_X_selected = (valid_X_selected  - np.mean(valid_X_selected , axis=0)) / np.std(valid_X_selected , axis=0)

        K = 5
        taus = [0.975,0.84,0.5,0.16,0.025]  # 1sigma = 68%
        # print(taus)
        targets = np.zeros((len(train_Y), K))
        for i in range(K):
            targets[:, i] = train_Y[:, 0]
        # print(targets.shape)


        xgb_new = xgb.XGBRegressor(objective=partial(arctan_loss, taus=taus, s=0.05),
                                base_score=0,
                                multi_strategy="multi_output_tree",
                                min_child_weight=0,
                                max_delta_step=.5)

        '''xgb_original = xgb.XGBRegressor(objective='reg:quantileerror', quantile_alpha=taus, base_score=0,
                                        min_child_weight=0)
        '''
        def get_best_model(model, X, Y, param_grid):
            wrapped_model = TransformedTargetRegressor(regressor=model, transformer=StandardScaler())
            grid_search = GridSearchCV(wrapped_model, param_grid, cv=5,
                                scoring=partial(scoring_loss, taus=taus))
            grid_search.fit(X, Y)
            return grid_search

        param_grid = {
                    'regressor__n_estimators': [200],#, 400],
                    'regressor__max_leaves': [400],
                    'regressor__max_depth': [3], 
                    'regressor__learning_rate': [0.05],
                    'regressor__gamma': [0.1],
                    'regressor__lambda': [1.0],
                }

        start_time2 = time()
        grid_search_new = get_best_model(xgb_new, train_X_selected, targets, param_grid)
        end_time2 = time()
        # print(end_time2-start_time2)




        spring_quantiles_2019 = grid_search_new.best_estimator_.predict(train_X_selected) 
        #print(spring_quantiles_2019.shape)

        import numpy as np
        from sklearn.metrics import r2_score

        y_true_spring_2019 = train_Y 
        y_pred_spring_2019 = spring_quantiles_2019[:,2] 

        y_true_spring_2019 = y_true_spring_2019.flatten()
        y_pred_spring_2019 = y_pred_spring_2019.flatten()

        #print(y_true_spring_2019)
        #print(y_pred_spring_2019)
        y_true_spring_2019_mean = np.mean(y_true_spring_2019)

        #print(y_true_spring_2019_mean)
        #print(len(y_true_spring_2019))
        #print(len(y_pred_spring_2019))
        true_train_cnt = 0
        TP = 0
        FN = 0
        FP = 0
        TN = 0
        for ti in range(len(y_true_spring_2019)):
            
            if y_true_spring_2019[ti]<y_true_spring_2019_mean and y_pred_spring_2019[ti]<y_true_spring_2019_mean:
                true_train_cnt = true_train_cnt + 1
                TP = TP + 1
            else:

                if y_true_spring_2019[ti]<y_true_spring_2019_mean and y_pred_spring_2019[ti]>y_true_spring_2019_mean:
                    FN = FN + 1

                if y_true_spring_2019[ti]>y_true_spring_2019_mean and y_pred_spring_2019[ti]<y_true_spring_2019_mean:
                    FP = FP + 1

                if y_true_spring_2019[ti]>y_true_spring_2019_mean and y_pred_spring_2019[ti]>y_true_spring_2019_mean:
                    true_train_cnt = true_train_cnt + 1
                    TN = TN + 1
        acc_score = float(true_train_cnt)/float(len(y_pred_spring_2019))
        Precision = float(TP) / float(TP+FP)
        Recall = float(TP) / float(TP+FN)
        F1 = (2*Precision*Recall) / (Precision+Recall) 

        acc_score_train.append(acc_score)
        Precision_train.append(Precision)
        Recall_train.append(Recall)
        F1_train.append(F1)



        fall_quantiles_2019 = grid_search_new.best_estimator_.predict(valid_X_selected) # 
        #print(fall_quantiles_2019.shape)

        import numpy as np
        from sklearn.metrics import r2_score

        y_true_fall_2019 = valid_Y #
        y_pred_fall_2019 = fall_quantiles_2019[:,2]  # 

        y_true_fall_2019 = y_true_fall_2019.flatten()
        y_pred_fall_2019 = y_pred_fall_2019.flatten()

        #print(y_true_fall_2019)
        #print(y_pred_fall_2019)
        y_true_fall_2019_mean = np.mean(y_true_fall_2019)
        #print(y_true_fall_2019_mean)
        #print(len(y_true_fall_2019))
        #print(len(y_pred_fall_2019))
        true_valid_cnt = 0
        TP = 0
        FN = 0
        FP = 0
        TN = 0
        for ti in range(len(y_true_fall_2019)):

            if y_true_fall_2019[ti]<y_true_fall_2019_mean and y_pred_fall_2019[ti]<y_true_fall_2019_mean:
                true_valid_cnt = true_valid_cnt + 1
                TP = TP + 1
            else:

                if y_true_fall_2019[ti]<y_true_fall_2019_mean and y_pred_fall_2019[ti]>y_true_fall_2019_mean:
                    FN = FN + 1

                if y_true_fall_2019[ti]>y_true_fall_2019_mean and y_pred_fall_2019[ti]<y_true_fall_2019_mean:
                    FP = FP + 1

                if y_true_fall_2019[ti]>y_true_fall_2019_mean and y_pred_fall_2019[ti]>y_true_fall_2019_mean:
                    true_valid_cnt = true_valid_cnt + 1
                    TN = TN + 1
        acc_score = float(true_valid_cnt)/float(len(y_pred_fall_2019))
        Precision = float(TP) / float(TP+FP)
        Recall = float(TP) / float(TP+FN)
        F1 = (2*Precision*Recall) / (Precision+Recall) 


        acc_score_valid.append(acc_score)
        Precision_valid.append(Precision)
        Recall_valid.append(Recall)
        F1_valid.append(F1)

    def mean(x):
        s = sum(x)
        l = len(x)
        return s/l


    print('acc_score_train',mean(acc_score_train))
    print('Precision_train',mean(Precision_train))
    print('ecall_train',mean(Recall_train))
    print('F1_train',mean(F1_train))

    print('acc_score_valid',mean(acc_score_valid))
    print('Precision_valid',mean(Precision_valid))
    print('Recall_valid',mean(Recall_valid))
    print('F1_valid',mean(F1_valid))


# fea 54
acc_score_train 0.8860000000000001
Precision_train 0.9133303037287647
ecall_train 0.8434341172079133
F1_train 0.876340776056502
acc_score_valid 0.6326530612244897
Precision_valid 0.6848123564276014
Recall_valid 0.5174434197042892
F1_valid 0.5673790166473094


In [6]:
print(train_X_selected.shape)
print(valid_X_selected.shape)
print(test_X_selected.shape)

(200, 54)
(49, 54)
(198, 54)


In [7]:
test_X_selected_0 = test_X_selected
print(np.min(test_X_selected))

-75.0


In [8]:
for dim in range(5, 6):
    #print('dim',dim, end=' ')
    test_X_selected = test_X_selected_0
    #print(np.min(test_X_selected))

    train_data_all = dataset[dataset['Group']=='Spring']
    train_all_X = train_data_all.drop(['SubjectID','Group','exam_avg'], axis=1).values
    train_all_Y = train_data_all[['exam_avg']].values

    #print(train_all_X.shape)
    #print(train_all_Y.shape)

    train_all_X_selected = selector.transform(train_all_X)

    train_all_X_selected = train_all_X_selected #+ 0.01   

    test_X_selected = test_X_selected #+ 0.01

    import numpy as np

    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score

    from TCA import TCA


    np.random.seed(42)
    K = 5
    taus = [0.975,0.84,0.5,0.16,0.025]  # 1sigma = 68%
    #print(taus)
    targets = np.zeros((len(train_all_Y), K))
    for i in range(K):
        targets[:, i] = train_all_Y[:, 0]
    #print(targets.shape)

    xgb_new = xgb.XGBRegressor(objective=partial(arctan_loss, taus=taus, s=0.05),
                            base_score=0,
                            multi_strategy="multi_output_tree",
                            min_child_weight=0,
                            max_delta_step=.5)

    '''xgb_original = xgb.XGBRegressor(objective='reg:quantileerror', quantile_alpha=taus, base_score=0,
                                    min_child_weight=0)
    '''
    def get_best_model(model, X, Y, param_grid):
        wrapped_model = TransformedTargetRegressor(regressor=model, transformer=StandardScaler())
        grid_search = GridSearchCV(wrapped_model, param_grid, cv=5,
                            scoring=partial(scoring_loss, taus=taus))
        grid_search.fit(X, Y)
        return grid_search

    param_grid = {
                'regressor__n_estimators': [200],#, 400],
                'regressor__max_leaves': [400],
                'regressor__max_depth': [3],  
                'regressor__learning_rate': [0.05],
                'regressor__gamma': [0.1],
                'regressor__lambda': [1.0],
            }

    start_time2 = time()
    grid_search_new = get_best_model(xgb_new, train_all_X_selected, targets, param_grid)
    end_time2 = time()
    #print(end_time2-start_time2)


    spring_quantiles_2019 = grid_search_new.best_estimator_.predict(train_all_X_selected)
    #print(spring_quantiles_2019.shape)

    import numpy as np
    from sklearn.metrics import r2_score

    y_true_spring_2019 = train_all_Y 
    y_pred_spring_2019 = spring_quantiles_2019[:,2] 

    y_true_spring_2019 = y_true_spring_2019.flatten()
    y_pred_spring_2019 = y_pred_spring_2019.flatten()

    #print(y_true_spring_2019)
    #print(y_pred_spring_2019)
    y_true_spring_2019_mean = np.mean(y_true_spring_2019)

    #print(y_true_spring_2019_mean)
    #print(len(y_true_spring_2019))
    #print(len(y_pred_spring_2019))
    true_train_cnt = 0
    TP = 0
    FN = 0
    FP = 0
    TN = 0
    for ti in range(len(y_true_spring_2019)):
        
        if y_true_spring_2019[ti]<y_true_spring_2019_mean and y_pred_spring_2019[ti]<y_true_spring_2019_mean:
            true_train_cnt = true_train_cnt + 1
            TP = TP + 1
        else:
            
            if y_true_spring_2019[ti]<y_true_spring_2019_mean and y_pred_spring_2019[ti]>y_true_spring_2019_mean:
                FN = FN + 1
            
            if y_true_spring_2019[ti]>y_true_spring_2019_mean and y_pred_spring_2019[ti]<y_true_spring_2019_mean:
                FP = FP + 1
            
            if y_true_spring_2019[ti]>y_true_spring_2019_mean and y_pred_spring_2019[ti]>y_true_spring_2019_mean:
                true_train_cnt = true_train_cnt + 1
                TN = TN + 1
    acc_score = float(true_train_cnt)/float(len(y_pred_spring_2019))
    Precision = float(TP) / float(TP+FP)
    Recall = float(TP) / float(TP+FN)
    F1 = (2*Precision*Recall) / (Precision+Recall) 

    print('train')
    print('Accuracy:', acc_score)
    print('Precision:',Precision)
    print('Recall:',Recall)
    print("F1:",F1)


    new_quantiles_2019 = grid_search_new.best_estimator_.predict(test_X_selected)
    #print(new_quantiles_2019.shape)
    #print(test_Y.shape)

    import numpy as np
    from sklearn.metrics import r2_score        
    y_true = test_Y 
    y_pred = new_quantiles_2019[:,2]  

    y_true = y_true.flatten()
    y_pred = y_pred.flatten()


    #print(y_true)
    #print(y_pred)
    y_true_mean = np.mean(y_true)

    #print(y_true_mean)
    #print(len(y_true))
    #print(len(y_pred))
    true_test_cnt = 0
    TP = 0
    FN = 0
    FP = 0
    TN = 0
    for ti in range(len(y_true)):
        
        if y_true[ti]<y_true_mean and y_pred[ti]<y_true_mean:
            true_test_cnt = true_test_cnt + 1
            TP = TP + 1
        else:
            
            if y_true[ti]<y_true_mean and y_pred[ti]>y_true_mean:
                FN = FN + 1
            
            if y_true[ti]>y_true_mean and y_pred[ti]<y_true_mean:
                FP = FP + 1

            if y_true[ti]>y_true_mean and y_pred[ti]>y_true_mean:
                true_test_cnt = true_test_cnt + 1
                TN = TN + 1
    acc_score = float(true_test_cnt)/float(len(y_pred))
    Precision = float(TP) / float(TP+FP)
    Recall = float(TP) / float(TP+FN)
    F1 = (2*Precision*Recall) / (Precision+Recall) 

    print('test')
    print('Accuracy:', acc_score)
    print('Precision:',Precision)
    print('Recall:',Recall)
    print("F1:",F1)

train
Accuracy: 0.8714859437751004
Precision: 0.93
Recall: 0.788135593220339
F1: 0.8532110091743119
test
Accuracy: 0.5858585858585859
Precision: 0.5283018867924528
Recall: 0.9230769230769231
F1: 0.6719999999999999


In [9]:
print(np.min(test_X_selected_0))

-75.0


In [10]:
for dim in range(20,21):
    print('dim',dim, end=' ')
    test_X_selected = test_X_selected_0
    #print(np.min(test_X_selected))

    train_data_all = dataset[dataset['Group']=='Spring']
    train_all_X = train_data_all.drop(['SubjectID','Group','exam_avg'], axis=1).values
    train_all_Y = train_data_all[['exam_avg']].values

    #print(train_all_X.shape)
    #print(train_all_Y.shape)

    train_all_X_selected = selector.transform(train_all_X)
    #print(train_all_X_selected.shape)

    train_all_X_selected = train_all_X_selected + 0.01   

    test_X_selected = test_X_selected + 0.01

    train_all_X_selected = (train_all_X_selected - np.mean(train_all_X_selected, axis=0)) / np.std(train_all_X_selected, axis=0)
    test_X_selected = (test_X_selected  - np.mean(test_X_selected , axis=0)) / np.std(test_X_selected , axis=0)

    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score

    from TCA import TCA

    np.random.seed(42)

    tca = TCA(kernel_type='linear', dim=dim, lamb=1, gamma=1)

    train_all_X_selected , test_X_selected = tca.fit(train_all_X_selected , test_X_selected)   #  tca.fit(X_source, X_target)

    K = 5
    taus = [0.975,0.84,0.5,0.16,0.025]  # 1sigma = 68%
    #print(taus)
    targets = np.zeros((len(train_all_Y), K))
    for i in range(K):
        targets[:, i] = train_all_Y[:, 0]
    #print(targets.shape)




    xgb_new = xgb.XGBRegressor(objective=partial(arctan_loss, taus=taus, s=0.05),
                            base_score=0,
                            multi_strategy="multi_output_tree",
                            min_child_weight=0,
                            max_delta_step=.5)

    '''xgb_original = xgb.XGBRegressor(objective='reg:quantileerror', quantile_alpha=taus, base_score=0,
                                    min_child_weight=0)
    '''
    def get_best_model(model, X, Y, param_grid):
        wrapped_model = TransformedTargetRegressor(regressor=model, transformer=StandardScaler())
        grid_search = GridSearchCV(wrapped_model, param_grid, cv=5,
                            scoring=partial(scoring_loss, taus=taus))
        grid_search.fit(X, Y)
        return grid_search

    param_grid = {
                'regressor__n_estimators': [200],#, 400],
                'regressor__max_leaves': [400],
                'regressor__max_depth': [3],  # 
                'regressor__learning_rate': [0.05],
                'regressor__gamma': [0.1],
                'regressor__lambda': [1.0],
            }

    start_time2 = time()
    grid_search_new = get_best_model(xgb_new, train_all_X_selected, targets, param_grid)
    end_time2 = time()
    #print(end_time2-start_time2)


    spring_quantiles_2019 = grid_search_new.best_estimator_.predict(train_all_X_selected)
    #print(spring_quantiles_2019.shape)

    import numpy as np
    from sklearn.metrics import r2_score

    y_true_spring_2019 = train_all_Y 
    y_pred_spring_2019 = spring_quantiles_2019[:,2] 

    y_true_spring_2019 = y_true_spring_2019.flatten()
    y_pred_spring_2019 = y_pred_spring_2019.flatten()

    #print(y_true_spring_2019)
    #print(y_pred_spring_2019)
    y_true_spring_2019_mean = np.mean(y_true_spring_2019)

    #print(y_true_spring_2019_mean)
    #print(len(y_true_spring_2019))
    #print(len(y_pred_spring_2019))
    true_train_cnt = 0
    TP = 0
    FN = 0
    FP = 0
    TN = 0
    for ti in range(len(y_true_spring_2019)):

        if y_true_spring_2019[ti]<y_true_spring_2019_mean and y_pred_spring_2019[ti]<y_true_spring_2019_mean:
            true_train_cnt = true_train_cnt + 1
            TP = TP + 1
        else:

            if y_true_spring_2019[ti]<y_true_spring_2019_mean and y_pred_spring_2019[ti]>y_true_spring_2019_mean:
                FN = FN + 1

            if y_true_spring_2019[ti]>y_true_spring_2019_mean and y_pred_spring_2019[ti]<y_true_spring_2019_mean:
                FP = FP + 1

            if y_true_spring_2019[ti]>y_true_spring_2019_mean and y_pred_spring_2019[ti]>y_true_spring_2019_mean:
                true_train_cnt = true_train_cnt + 1
                TN = TN + 1
    acc_score = float(true_train_cnt)/float(len(y_pred_spring_2019))
    Precision = float(TP) / float(TP+FP)
    Recall = float(TP) / float(TP+FN)
    F1 = (2*Precision*Recall) / (Precision+Recall) 

    print('train')
    print('Accuracy:', acc_score)
    print('Precision:',Precision)
    print('Recall:',Recall)
    print("F1:",F1)


    new_quantiles_2019 = grid_search_new.best_estimator_.predict(test_X_selected)
    #print(new_quantiles_2019.shape)
    #print(test_Y.shape)

    import numpy as np
    from sklearn.metrics import r2_score     
    y_true = test_Y  
    y_pred = new_quantiles_2019[:,2] 

    y_true = y_true.flatten()
    y_pred = y_pred.flatten()


    #print(y_true)
    #print(y_pred)
    y_true_mean = np.mean(y_true)

    #print(y_true_mean)
    #print(len(y_true))
    #print(len(y_pred))
    true_test_cnt = 0
    TP = 0
    FN = 0
    FP = 0
    TN = 0
    for ti in range(len(y_true)):
        if y_true[ti]<y_true_mean and y_pred[ti]<y_true_mean:
            true_test_cnt = true_test_cnt + 1
            TP = TP + 1
        else:

            if y_true[ti]<y_true_mean and y_pred[ti]>y_true_mean:
                FN = FN + 1

            if y_true[ti]>y_true_mean and y_pred[ti]<y_true_mean:
                FP = FP + 1

            if y_true[ti]>y_true_mean and y_pred[ti]>y_true_mean:
                true_test_cnt = true_test_cnt + 1
                TN = TN + 1
    acc_score = float(true_test_cnt)/float(len(y_pred))
    Precision = float(TP) / float(TP+FP)
    Recall = float(TP) / float(TP+FN)
    F1 = (2*Precision*Recall) / (Precision+Recall) 

    print('test')
    print('Accuracy:', acc_score)
    print('Precision:',Precision)
    print('Recall:',Recall)
    print("F1:",F1)

dim 20 train
Accuracy: 0.8714859437751004
Precision: 0.8583333333333333
Recall: 0.8728813559322034
F1: 0.865546218487395
test
Accuracy: 0.5909090909090909
Precision: 0.5316455696202531
Recall: 0.9230769230769231
F1: 0.6746987951807228
